## Import library and loading data

In [3]:
import nltk
import re
import numpy as np
import sklearn
import gensim
import string

In [4]:
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup
from gensim import corpora, models, similarities
from sklearn.model_selection import train_test_split

In [5]:
from replacers import RepeatReplacer
from replacers import RegexpReplacer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
wordnet_lemmatizer = WordNetLemmatizer()
replacer=RegexpReplacer()
repeat= RepeatReplacer()

In [6]:
# Load positive reviews
positive_reviews = BeautifulSoup(open('electronics/positive.review').read())
positive_reviews = positive_reviews.findAll('review_text')

C:\Users\s4341237\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\s4341237\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [7]:
# Load negative reviews
negative_reviews = BeautifulSoup(open('electronics/negative.review').read())
negative_reviews = negative_reviews.findAll('review_text')

C:\Users\s4341237\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\s4341237\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


## Preprocessing Word

In [37]:
def my_tokenizer(s):
    replacer=RegexpReplacer()
    repeat= RepeatReplacer()
    stops=set(stopwords.words('english'))
    s = s.lower()
    tokens = word_tokenize(replacer.replace(s))
    tokens = word_tokenize(repeat.replace(s))
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
    #tokens = [stemmer.stem(t) for t in tokens]
    #tokens = [t for t in tokens if t not in stops]
    tokens = [t for t in tokens if t not in string.punctuation]
    return tokens

In [38]:
word_index_map = {}
current_index = 0

In [39]:
positive_tokenized = []
negative_tokenized = []

In [40]:
for review in positive_reviews:
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [41]:
for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [42]:
# N is the size of numpy array which contents all the vector of each review
N = len(positive_tokenized) + len(negative_tokenized)
length_dict = len(word_index_map)
#data = np.zeros((N, len(word_index_map) + 1))
data = np.zeros((N, (length_dict * 50) + 1))

In [43]:
sum_tokenized = positive_tokenized + negative_tokenized
model = gensim.models.Word2Vec(sum_tokenized, min_count=1, size= 50)

In [44]:
def tokens_to_vector(tokens, label):
    x = np.zeros((len(word_index_map),50))
    for t in tokens:
        if t in word_index_map:
            i = word_index_map[t]
            x[i] = model[t]
        else:
            x.fill(0)            
    x = np.reshape(x, (len(word_index_map)*50))
    x = np.append(x, label)
    return x

In [45]:
i = 0 # Set counter to 0
# Convent each group of token from each reviews
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i += 1

In [46]:
# Convert each group of token from each reviews
for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i += 1

In [47]:
# Shuffle data for training 
np.random.shuffle(data)

## Training the model

In [48]:
X = data[:, :-1]    # all the row and the first 11093 column
Y = data[:, -1]     # all the row The last column

In [49]:
#Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.5, random_state=3)
Xtrain = X[:-100,]
Ytrain = Y[:-100,]
Xtest = X[-100:,]
Ytest = Y[-100:,]

#### Logistic Regression 

In [50]:
LR = LogisticRegression()
LR.fit(Xtrain, Ytrain)
print("Logistic Regression classification rate:", LR.score(Xtest, Ytest))

Logistic Regression classification rate: 0.76


In [23]:
for word, index in word_index_map.items():
    weight = LR.coef_[0][index]
    print(word, weight)

ever-present 0.0119689418981
abilty 0.00989491121912
mb/s 0.00337512748681
20th -0.00269581191966
deleting 0.243932299287
comercial -0.031258334288
taylor 0.0447129136835
pleasing -0.00210911306155
obsesed 0.0566911587952
ragedy 0.016961407486
lastest -0.00175539362747
include -0.00443758519496
asurity 0.0345205826773
signels 0.0251384112795
novice 0.0481834922891
dvi-d -0.0034526170723
i.e. -7.44920980676e-07
ofen 0.00491671479394
doa 0.0296400428963
108m 0.0289080564012
720ul -0.113781859608
acustic 0.0144699561248
configure 0.00531166044973
'mac -0.000242215507617
eventhough 0.0164664872047
previously 0.00421792120943
senheiser 0.016051002867
certain 0.00785655249522
cnd -1.24958871539e-05
dvd740e -0.00170645605081
1.2.0 0.0187263586096
h215 -0.0112197666668
input/output-not 0.00130180116378
wedings 0.00867030346294
man-handling 0.0125627561929
times..but..i -0.00434479913327
wlancfg5 0.000420816232087
sued 0.0353125503877
merely 0.0147236300631
rechargable 0.00844665246666
compatab

1973 -0.000816135505003
routers/aces 0.00991349770447
mia -0.00356763484528
half-*** 0.0182537644815
buzed 0.0399508113512
turning -0.00600633683717
auction 0.00724091335941
fv50 0.140832814585
eliminate 0.0108206056291
clap -0.00769905675857
sweatpants 0.0294764144278
mvs3-1m -0.000283448226005
at-65 -0.000234740842383
fighting 0.105590506198
crep 0.0160382831328
awaits -0.0100632513229
mark 0.000401919139309
heath -0.0481865403231
yelow.. 0.00501901474805
2-50 -0.000211859070056
unpack -0.100564424336
powerjolt 0.0152627792179
delphi 0.0711887414025
mute/unmute 0.0239521077218
cup-holder -0.0129042038418
mouse-to-mouse 0.0071966219487
threaten -0.022116674196
z-5450 0.0306845424509
sophisticated 0.0133201434446
managed 0.00539356947856
delicate 0.0413379877296
superoposterior -0.027280888975
101 0.0515153290029
number 0.0189342712621
burner 0.0582472054946
finlay 0.0119716209977
tired 0.074385467638
bounce 0.0051802449949
drawing 0.00469277853348
3.1 0.00246337179744
recomended -0.00

8.1 -0.0125273599396
overloked -0.0109182723282
multi -0.0930760165812
noise-canceling -0.00823052699405
road-trip 0.0371672183207
teribl 0.0275784441862
650. 0.0460203501471
please 0.032562517614
lately -0.00879491065406
zd7050 -0.0306944426632
deduction 0.0148367646942
dt20v -0.00468563473674
locked 0.00157592187575
frequencies..rated -0.0065118643615
hls-5687w -0.00282559192122
dj -0.00499714105907
existance -0.0211961246339
back 0.0033548920624
knocking 0.00394820795416
iritating 0.0739681523291
spend -0.0187696986594
pine 0.013946594711
everyday -0.00608204971821
non-direction 0.00103438078777
curopt 0.0400736387837
recharges -0.0107398631015
ibok 0.0089424589969
register -0.00916626725211
esta 0.02036018427
daxon016s -0.00276950831741
bilion 0.0145485105606
outweighs -0.0803113659963
108mbps 0.0607221682082
polk -0.00398176977768
droping -0.00412487750492
pod -0.00868931577984
excelecnt 0.00958243460063
afternon -0.0253091850541
24.0 0.0430232585781
extorted -0.0358634557326
main

49 -0.0147061932617
one-buton 0.020506002297
wpc54g -0.0323464706388
local -0.00715315104715
macbok 0.110249331756
computer-saturated -0.0342141015408
unclear 0.00268924783291
certification -0.00024969160113
march 0.0105888445756
almost 0.00414117411385
comparably -0.00108739307413
re-routes 0.0124080679098
background -0.000572197764834
everytime 0.0126088262396
integrate 0.0291387262843
pause 2.45182515617e-08
bulb 0.0131076315239
ready -0.000777834626394
waterprof -0.0389716801691
maping 0.0143245363605
simulated 0.067707368212
pleasantly 0.00294033333039
magic -0.00194311387822
expedition 0.219677512913
online- -0.00017972833347
spekaers 0.0048590584408
av 0.0336941341798
odysey 0.0120612842677
203 -0.0298739334037
scrols -0.00989528117517
deluxe -0.000171721134352
afortable -0.00673357012126
seler 0.00191344928623
lifesaver 0.0593285919897
acounstic 0.00790803933468
visible 0.000510704854743
rank 0.0785452649049
lamp -0.0506686137746
roadblock -0.0122939466686
horn -0.0009911824259

homework -0.0183253708677
lops 0.0154278710875
apologize -0.0026476574217
530 0.144384800335
degred -0.0169711213737
suces -0.0118438326773
miserable -0.00160961106097
specificaly -0.0204486159767
340.0 0.002287577027
reinserted 0.0258891193982
scansnap 0.00103470402527
safety 0.0015762969391
spoken -0.00563901867544
put 0.0340144349745
50cent -0.014048663219
snapier -0.00163496594889
cartidges -0.00424567644592
researching -0.0330175006589
loyal -0.143590269298
pen 0.0187161043764
intensified 0.0130717554517
tco -0.00423549884311
ur-20 0.00879256860226
bird -0.132669314287
complex 0.00322402330393
faraway 0.0801402740089
680gt 0.0164046579777
magical 0.00688545683168
120 -0.00478439032819
justify -0.00044444373769
bateries -0.00125533008196
low-cost 0.000864688964326
non-microsoft -5.2730956983e-05
cord/construction 0.00324862500717
net1.exe 0.000967484923401
dvd-ram 0.0080247077039
atach -0.00100187460439
sturdily -0.00975153441147
inflict 0.0482602388009
city-by-city -0.052138830145

...

1280x1024 0.0400465383318
fewer 0.0154953847818
invalidate -0.0181380923412
reconected 0.0607943637004
lugage -0.00242817910327
facility 0.195616786679
480 0.00071614151021
thet -0.0748258495022
double-sided -0.00483477757849
streamlined 0.00620651345084
fail 0.00260710364862
unhok 0.000227010617648
screwdriver -0.0404640139099
'photo 0.0163988061314
thereof 0.0144212520994
shelf 0.0508360140545
hiden 0.0340341717437
peripheral 0.0140006645776
sobering -0.000649774424551
e260 -0.0123868356956
bumer 0.0237890151606
saver 0.0016650714878
adapter 0.0406717845986
6-69 -0.00084773925699
comparied -0.117503103386
radar -0.00586969528888
describing -0.000527937710717
worthwhile -0.0162202194299
enable 0.00693575328592
presure -0.0373802159582
swinging 0.0333370200793
puchased 0.0584861459354
.stay 0.0168758956573
disconects -0.00278291287337
rols -0.0371440725267
gen -0.00200710610276
similarly 0.117073050788
relief 0.0359369189027
shaking 0.00410874101687
g5 -0.0405919436439
192.168.0.10 -0.

nav -0.016748306434
italy -0.012725142404
selected 0.0292600268029
d. -0.0121191834437
ceo -2.17507407382e-05
smothed -0.015623261853
superior -0.0135107070225
grainines 0.000229646783375
tough 0.0227091406782
kx-tg576m -0.0347563916987
consulting 0.00294354720454
asap 0.00136658907246
eject -0.0365606915476
software 0.05812090956
impedes -0.0784291902979
financial 0.0249142609152
mi 0.0307097289095
destructs -0.000419218987048
unplug/plug 0.00561077374515
minumum -0.0722214153287
re-enforced 0.0127324017624
ergonomics -0.0141876381182
liner -0.0454353330652
5 -0.037318737801
preferable -0.00144030138177
acting -0.0307236190562
bedrom 0.00614206636147
360 -0.0181500369921
transit/storage 0.000832551928211
release 0.0549209509192
statement -0.0135882445739
eforts 9.35000768601e-05
advised -0.000702656326531
dvdrs -0.00647142692226
minimal -0.0134908862834
dreading -0.0111926680096
played -0.00189542831609
frils -0.0884298756525
pertains -0.0240443825674
terible -0.0198998995166
confront

videophile -0.0588590497888
contrary -0.0245146475005
dvds.. -0.00816592029886
taped -0.0271479282715
super-expensive 0.000722145237061
nearly -0.00463843162942
maker -9.4304774293e-08
fetish 0.000879044774849
muscle 0.0250382958028
muc 0.00824715492727
..a -0.00558986474457
clearly -0.00816132522538
o. -0.0215665281585
laptop/desktop -0.0424951079098
improvement -0.0106207457111
products.however 0.0147032955305
circle 0.0138882745422
non-oem 0.151984304312
brand -0.00531446928339
adjustability -0.0123197041716
would -0.00572473255181
glue -0.0217039323431
unfriendly -0.0840343877466
efusive 0.0183585812084
h10 -0.00180166099221
sony -0.00588551597717
pitch 0.0630680558685
standard -0.000693153242658
aproximately -0.000361642253622
intal -0.0208019907504
yesterday 0.00196764291232
sounding 0.018712746406
wap5ag -0.000900286876191
cycled 0.0101062185903
cheapo 0.0133266366025
dvpns50/b -0.022488500308
english 0.0241110912959
wildly 0.0893906262246
t570 0.0431342091942
atatched -0.005069

malfunctioning -0.0114347068442
circuit 0.00141145393576
anyway -0.0579446312896
reataches 0.045568844265
displaying -0.0606090096536
phoned -0.00100119808959
apleworks -0.0414430200324
legitimate 0.0071683035173
resistance -0.030209344002
afraid -0.0217411531629
pavilion 0.00390079345269
aired 3.74624528724e-05
tad -0.00297054100646
fate 0.00580624472014
calculating 0.0115067680773
nomad -0.00439330821093
piano 5.20483585565e-08
remarkable 0.00909468125761
downfals 0.12547047077
non-earbud 0.0144391891628
x-fi 0.00350347584454
rapid 0.0523953086189
speakers-aestheticaly -0.0388265249085
grunge 0.0117619162629
multiple -0.0382774478536
beast 0.0768997300413
single-sided -0.00427952185614
kitin -0.0522928137744
mechanical -0.00526127372952
wind 0.0158622211895
zipers 0.00125110103232
*before* 0.0138784563817
linux -0.00230941008352
oval -0.0260505111764
longest 0.00612991494341
sensor -0.0121525931573
rich 0.0034417323869
hair 0.0102643348377
exicted 1.99797169357e-07
scroling -0.012256

shuter -0.00849104739122
enginers 0.0278790817055
9510i -0.00614244874873
yearly.. -0.00968943789924
bathrom 0.0524905839036
reliability/quality -0.000308750248044
17 0.0195951192992
problem-so -0.00434246968062
desire 0.00118575484157
rock-botom 0.0522421276907
10-cup -0.0142620947184
phrase 0.0105702308064
jitered -0.00242985505754
blog -0.00182332825235
adjusted 0.0320015184084
simple 0.0176940012578
bed -0.00220960023449
mhz 0.00815189207202
maneuver 0.0326034891439
al-in -0.00554771573861
flakey -0.0114804096865
face 0.0781395446426
net 0.00418459773007
ir -0.000194097959109
bluray 0.0112889717539
centered -0.108760891481
15-minute -0.121258131627
tremendo -0.00825082828892
alphabet -0.0375024059979
cox 0.149185201683
protective 0.0945012922443
close 0.00225879664041
jumpdrive 0.0213711334004
wenger 0.0475635266833
ridled -0.023362990083
30-35 -0.0059630666506
cablevision 0.00126791742207
qam 0.0113967223286
caled-parties 0.0173077379159
tightened -0.0186480849757
recalculating 0.

#### Naive Bayes

In [24]:
from sklearn import svm

In [25]:
SVM = svm.LinearSVC()
SVM.fit(Xtrain, Ytrain)
print("SVN classification rate:", SVM.score(Xtest, Ytest))

SVN classification rate: 0.7


#### Decision Tree

In [26]:
from sklearn import tree

In [27]:
DT = tree.DecisionTreeClassifier()
DT.fit(Xtrain, Ytrain)
print("Decision Tree classification rate:", DT.score(Xtest, Ytest))

Decision Tree classification rate: 0.7


## Test with real data

In [51]:
string_text_negative = "Not stable. Had to return."
#string_text_negative = Not happy... very sloppy made others were better. I have received better made fans for less. They made these very sloppy some were broken or stuck with glue."

In [52]:
string_text_positive = "Yes I liked this product.  Horror Hotel is one of my fav classic horror movies  Finding it along with another horror movie was a plus.  Very good value, arrived within 5 days."

In [55]:
string_token = my_tokenizer("I hate it")
vector_test = tokens_to_vector(string_token, 1)
array_string_test = vector_test[:-1]

In [56]:
LR.predict([array_string_test])

array([ 1.])

In [59]:
word_index_map['it']
LR.coef_[0][1968]

-0.00085993647228223508

##  New Tokenize function

In [18]:
from replacers import RepeatReplacer
from replacers import RegexpReplacer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [14]:
def my_tokenizer_new(s):
    replacer=RegexpReplacer()
    repeat= RepeatReplacer()
    stops=set(stopwords.words('english'))
    s = s.lower()
    tokens = word_tokenize(replacer.replace(s))
    tokens = word_tokenize(repeat.replace(s))
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
    #tokens = [stemmer.stem(t) for t in tokens]
    tokens = [t for t in tokens if t not in stops]
    tokens = [t for t in tokens if t not in string.punctuation]
    return tokens

In [16]:
my_tokenizer_new("the buses is going")

['bus', 'going']

In [16]:
import re
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
def tokenizer(text):
    text = re.sub('<[^>]*>','',text)
    smileys = re.findall('((?::|;|=)(?:-?)(?:[D|d|)|(|P|p|/|x|X]))',text)
    text = re.sub('[\W]+',' ',text.lower())
    text += ' '.join(smileys).replace('-','')
    tokenized = [w for w in text.split() if w not in stop]
    #return " ".join(tokenized)
    return tokenized

In [123]:
tokenizer("2-T, 123f, I'm Vu, 12345, !@#$$%%^&~, :-), :-|")

['2', '123f', 'vu', '12345', ':)', ':|']

In [131]:
from nltk.stem import PorterStemmer

In [132]:
stemmer = PorterStemmer()

In [160]:
stemmer.stem('believer')

'believ'

In [164]:
wordnet_lemmatizer.lemmatize('going')

'going'

In [13]:
replacer = RepeatReplacer()
replacer.replace('zoo')

'zoo'

ImportError: cannot import name 'SpellingReplacer'